##Implementation of LesNet50

In [1]:
import torch #importing necessary libraries
import torch.nn as nn
import torchvision
from torchvision.ops import nms
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim

import torchvision.transforms as transforms

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)        # C1
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2) # S2
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)       # C3
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2) # S4
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5)     # C5
        self.fc1 = nn.Linear(120, 84)                      # F6
        self.fc2 = nn.Linear(84, num_classes)              # Output

#defining how input flows through the neural network using forward in pytorch
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)  # Flatten, must be flattened before fully connected layers as they accept input as only 1d vector.
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [3]:
#importing MNIST dataset to test LeNet5 for classification

transform = transforms.Compose([transforms.Pad(2),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [5]:
num_epochs = 10  # Try 2 epochs for a quick test

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Epoch {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')


Epoch 1 [0/60000] Loss: 2.306676
Epoch 1 [6400/60000] Loss: 0.419217
Epoch 1 [12800/60000] Loss: 0.402976
Epoch 1 [19200/60000] Loss: 0.229081
Epoch 1 [25600/60000] Loss: 0.130060
Epoch 1 [32000/60000] Loss: 0.133388
Epoch 1 [38400/60000] Loss: 0.118812
Epoch 1 [44800/60000] Loss: 0.081738
Epoch 1 [51200/60000] Loss: 0.064917
Epoch 1 [57600/60000] Loss: 0.100657
Epoch 2 [0/60000] Loss: 0.116946
Epoch 2 [6400/60000] Loss: 0.176799
Epoch 2 [12800/60000] Loss: 0.051048
Epoch 2 [19200/60000] Loss: 0.135149
Epoch 2 [25600/60000] Loss: 0.048388
Epoch 2 [32000/60000] Loss: 0.099510
Epoch 2 [38400/60000] Loss: 0.073915
Epoch 2 [44800/60000] Loss: 0.009971
Epoch 2 [51200/60000] Loss: 0.071799
Epoch 2 [57600/60000] Loss: 0.031409
Epoch 3 [0/60000] Loss: 0.084113
Epoch 3 [6400/60000] Loss: 0.025427
Epoch 3 [12800/60000] Loss: 0.038712
Epoch 3 [19200/60000] Loss: 0.014982
Epoch 3 [25600/60000] Loss: 0.036547
Epoch 3 [32000/60000] Loss: 0.022317
Epoch 3 [38400/60000] Loss: 0.027768
Epoch 3 [44800/6

In [6]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        
print(f'Test Accuracy: {100 * correct / total:.2f}%')


Test Accuracy: 99.04%
